## 1. Run gitter on all images

The gitter script consists of only two lines:
```
library("gitter")
#gitter.batch(".", plate.format=1536, inverse="TRUE", remove.noise="TRUE", grid.save="grid_images", dat.save="dat_files")
```

In [1]:
#%cd 20181216_assayPlates_2d/
#!Rscript gitter_script.R > gitter_log.txt
#%cd ..

[Errno 2] No such file or directory: '20181216_assayPlates_2d/'
/Users/aulakhs/Documents/RalserLab/metallica/experiment_data/metdep_KOgrowth/pyphe
/Users/aulakhs/Documents/RalserLab/metallica/experiment_data/metdep_KOgrowth


The .dat files produced by gitter are found in ./20181216_assayPlates_2d/dat_files/

## 2. Run pyphe-analyse for grid normalisation and data aggregation

The EDT file, which contains all relevant plate metadata, was created separately and looks as follows

In [ ]:
import pandas as pd
edt = pd.read_csv('pyphe/20190122_exp_data_2d.csv', index_col=0)
edt

In [ ]:
!pyphe-analyse --edt 20190122_exp_data_2d.csv --format gitter --out pyphe-analyse-report.csv --gridnorm standard1536 --extrapolate_corners --rcmedian --qc_plots qc_plots 



## 3. Add layout information

In [ ]:
### load pyphe-analyse data report
ld = pd.read_csv('pyphe/pyphe-analyse-report.csv', index_col=0)

ld['Assay_plate_position'] = ld['Layout'] + '_' + ld['Row'].astype(str) + '_' + ld['Column'].astype(str)

#Remove columns redundant with layout table
ld = ld.drop(['Layout', 'Row', 'Column', 'Arrangement'], axis=1)

### Load layout
layout = pd.read_csv('pyphe/20190122_screenArrangementLayout.csv', index_col=0)
ld = ld.join(layout, on='Assay_plate_position')

ld

## 4. perform QC checks on the data

In [ ]:
### check footprints (should be empty)
from matplotlib import pyplot as plt
import seaborn as sns
from collections import Counter

footprints = ld.loc[pd.isnull(ld['ID'])]
print(len(footprints.index))
contaminations = footprints.loc[footprints['Colony_size']>0]
contaminations[['Comment', 'Colony_size']]
print(len(contaminations.index))

fig, ax = plt.subplots(figsize=(10,3))

plate_counts = pd.Series(Counter(contaminations['Plate']))
plate_counts.plot(kind='bar', ax=ax)
ax.set_xlabel('Source plate')
ax.set_ylabel('number of contaminated footprints')

#out of 4875 footprints, only 49 are contaminated (~1%), in almost all plates where a footprint was contaminated only one position was contaminated, indicating no systematic error
#Plate dI_K50_2 will have to be excluded, had 13 wrong footprints


In [ ]:
#3. Checking the efficiency of grid correction with control grid
from scipy.stats import linregress
sns.set(font_scale=1.3, style='white')

fig, ax = plt.subplots(1, 3, figsize=(13,4), gridspec_kw={'width_ratios':[1.7, 1, 3]})

extragrid_ld = ld.loc[(ld['Cryostock_96_Plate']=='Pextragrid')]
print(len(extragrid_ld.index))
sns.regplot('Reference_surface', 'Colony_size', data=extragrid_ld, marker='o', scatter_kws={'facecolors':'none', 'alpha':0.5}, ax=ax[0])
ax[0].plot([0,2500], [0,2500], color='r')
corr = extragrid_ld[['Reference_surface','Colony_size']].corr().loc['Reference_surface','Colony_size']
ax[0].set_title('r=%.3f'%corr)

cspqc = extragrid_ld['Colony_size_corr_checked'].dropna()
sns.kdeplot(cspqc, ax=ax[1], clip=(0,2), legend=False)
print(len(cspqc.index), cspqc.mean(), cspqc.std())
ax[1].set_title('%.3f +/- %.3f'%(cspqc.mean(), cspqc.std()))
ax[1].set_xlabel('Corrected colony size \n of extra grid')
ax[1].set_ylabel('Density')


pos_prec = pd.DataFrame(index=range(1,33), columns=range(1,49), dtype=float)
pos_prec.update(extragrid_ld.pivot_table(index='Assayplate_row', columns='Assayplate_col', values='Colony_size_corr_checked'))  
sns.heatmap(pos_prec, ax=ax[2])
ax[2].set_title('Mean corrected colony size of extra grid')
ax[2].set_xlabel('Row')
ax[2].set_ylabel('Column')

plt.tight_layout()


In [ ]:
#4. Checking the technical correlation of within-plate repeats
reps = ld.loc[(ld['Arrangement_multiplex_position']=='A2') | (ld['Arrangement_multiplex_position']=='C1')]
reps_uncorr = reps.pivot_table(index='ID', columns='Arrangement_multiplex_position', values='Colony_size')
reps_corr = reps.pivot_table(index='ID', columns='Arrangement_multiplex_position', values='Colony_size_corr_checked')

fig, ax = plt.subplots(1,2, figsize=(5*1.618, 4))

sns.regplot('A2', 'C1', data=reps_uncorr, marker='o', scatter_kws={'facecolors':'none', 'alpha':0.5}, ax=ax[0])
ax[0].set_title('Raw colony size, r=0.78')
ax[0].set_xlabel('Repeat 1 (A2)')
ax[0].set_ylabel('Repeat 2 (C1)')

print(reps_uncorr.corr())

sns.regplot('A2', 'C1', data=reps_corr, marker='o', scatter_kws={'facecolors':'none', 'alpha':0.5}, ax=ax[1])
ax[1].set_xlim((0,2.5))
ax[1].set_ylim((0,2.5))
ax[1].set_title('Corrected colony size, r=0.93')
ax[1].set_xlabel('Repeat 1 (A2)')
ax[1].set_ylabel('Repeat 2 (C1)')
print(reps_corr.corr())

plt.tight_layout()


In [ ]:
### exclude some data based on manual inspection
import numpy as np

#copy the corrected fitness data for further manual cleaning
ld['Colony_size_corr_postQC'] = ld['Colony_size_corr_checked'].copy()

print("shape before filters:")
print(ld.shape)

#Exclude dI_K50 plate , all footprints are wrong
print(ld['Colony_size_corr_postQC'].isna().sum())
ld.loc[ld['Plate']=='dI_K50_2', 'Colony_size_corr_postQC'] = np.nan
print(ld['Colony_size_corr_postQC'].isna().sum())

#aV (all conditions!!), row 20-28, & col 14-21 (smudged colonies on combinedPlate)
ld.loc[(ld['Layout']=='aV') & (ld['Assayplate_col']>13) & (ld['Assayplate_col']<22) 
                & (ld['Assayplate_row']>19) & (ld['Assayplate_row']<29), 'Colony_size_corr_postQC'] = np.nan
print(ld['Colony_size_corr_postQC'].isna().sum())

#bIII_K20: row 3-5 & col 45-48 (inclusive) (smudged colonies), both timepoints!
ld.loc[(ld['Layout']=='bIII') & (ld['Condition']=='K20')
                & (ld['Assayplate_col']>44) & (ld['Assayplate_col']<49) 
                & (ld['Assayplate_row']>2) & (ld['Assayplate_row']<6), 'Colony_size_corr_postQC'] = np.nan
print(ld['Colony_size_corr_postQC'].isna().sum())

#bII_Ca_2: row 32, col 36 (obscuring object)
ld.loc[(ld['Plate']=='bII_Ca_2') 
                & (ld['Assayplate_col']==36) & (ld['Assayplate_row']==32), 'Colony_size_corr_postQC'] = np.nan
print(ld['Colony_size_corr_postQC'].isna().sum())

#bII_Zn1000 row 1-10, col 38-48 (weird strong growth in corner, media not well mixed?)
ld.loc[(ld['Layout']=='bII') & (ld['Condition']=='Zn1000')
                & (ld['Assayplate_col']>37) & (ld['Assayplate_col']<49) 
                & (ld['Assayplate_row']>0) & (ld['Assayplate_row']<11), 'Colony_size_corr_postQC'] = np.nan
print(ld['Colony_size_corr_postQC'].isna().sum())


#export final ld
ld.to_csv('pyphe-analyse-report_postQC.csv')

print("shape after filters:")
print(ld.shape)

## 5. Run pyphe-interpret

In [ ]:
!pyphe-interpret --ld pyphe-analyse-report_postQC.csv --out pyphe-interpret --grouping_column ID --axis_column Condition --values_column Colony_size_corr_postQC --control AE --circularity 0.9 --set_missing_na 


## 6. quick checks on the hits

In [ ]:
hd = pd.read_csv('pyphe-interpret_summaryStats.csv', header=[0,1], index_col=0)
hd.index.name = 'Gene'
hd.columns = hd.columns.set_names(['Condition',''])
hd

In [ ]:
#check if the FDR correction is correct
#confirms that correction for testing of multiple genes was done separately for each condition
from statsmodels.stats.multitest import fdrcorrection
fdr_check = hd[('K20','p_Welch')].copy().dropna()
fdr_check = pd.Series(fdrcorrection(fdr_check)[1], index=fdr_check.index)
fdr_check = pd.concat({'manual':fdr_check, 'pyphe':hd[('K20','p_Welch_BH')].dropna()}, axis=1)
print((fdr_check['manual']-fdr_check['pyphe']).abs().max()) #some numerical inprecision is allowed
fdr_check

In [ ]:
#rearrange table layout
hd.columns = hd.columns.swaplevel()
hd = hd.stack()
hd

In [ ]:
hd['is_hit'] = (hd['p_Welch_BH'] < 0.05) & (hd['median_effect_size_log2'].abs() > 0.25)
hd

In [ ]:
hd.groupby('Condition')['is_hit'].sum()